In [1]:
!pip install PyMuPDF langchain langchain-community langchain-openai langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: typing-e

In [7]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 4.5 MB/s eta 0:00:00


In [2]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 5.7 MB/s eta 0:00:00


In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/EN-05-10035.pdf")
pages = loader.load_and_split()

In [4]:
pages[0]

Document(page_content='Retirement Benefits2024\nSSA.gov', metadata={'source': '/content/EN-05-10035.pdf', 'page': 0})

In [5]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

In [ ]:
os.environ["PINECONE_API_KEY"] = "YOUR_API_KEY"
os.environ["PINECONE_INDEX_NAME"] = "YOUR_INDEX_NAME"

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

embeddings = OpenAIEmbeddings()

In [22]:
!pip install -U langchain-pinecone

ERROR: Could not find a version that satisfies the requirement langchain-pinecone (from versions: none)
ERROR: No matching distribution found for langchain-pinecone


In [24]:
!git clone https://github.com/langchain-ai/langchain.git

Cloning into 'langchain'...
remote: Enumerating objects: 128939, done.
remote: Counting objects: 100% (23094/23094), done.
remote: Compressing objects: 100% (1619/1619), done.
remote: Total 128939 (delta 22167), reused 21475 (delta 21475), pack-reused 105845
Receiving objects: 100% (128939/128939), 169.97 MiB | 26.78 MiB/s, done.
Resolving deltas: 100% (95056/95056), done.
Updating files: 100% (6361/6361), done.


In [26]:
cd /content/langchain/libs/partners/pinecone

/content/langchain/libs/partners/pinecone


In [ ]:
!pip install .

In [ ]:
cd

In [30]:
cd /content

/content


In [31]:
!export PINECONE_API_KEY="PINECONE_KEY"

In [32]:
!export PINECONE_INDEX_NAME="PINECONE_INDEX"

In [36]:
from pinecone import Pinecone as PineconeClient

In [38]:
client = PineconeClient(api_key="PINECONE_API")

indexes = client.list_indexes()

In [39]:
indexes

{'indexes': [{'dimension': 1536,
              'host': 'convaiid-g69j51d.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'convaiid',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [40]:
from langchain_pinecone import Pinecone

pinecone  = Pinecone(
    pinecone_api_key="PINECONE_API",
    index_name="PINECONE_INDEX",
    embedding = embeddings
)

In [42]:
for text in docs:
  pinecone.add_texts([text.page_content])

In [ ]:
retriever = pinecone.as_retriever(search_type="mmr")
matched_docs = retriever.get_relevant_documents("YOUR_QUERY")
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)

In [ ]:
""